In [1]:
# Create monthly report url

# To run this directly from the command line, 
# jupyter nbconvert --execute "USDA Monthly Report.ipynb" --to asciidoc

from datetime import datetime, timedelta, date
import urllib.parse

report_date = datetime.now()

# end = datetime.strptime(report_date, '%m/%d/%Y')
start = date(report_date.year, 1, 1)

donatedFoodValue = 127206.12

import config
base_url = config.church_domain_url + r"/events/reports/243292484#/"

import urllib.parse
params = {
    'start_date': urllib.parse.quote(start.strftime('%m/%d/%Y'), safe='/'),
    'end_date': urllib.parse.quote(report_date.strftime('%m/%d/%Y'), safe='/'),
    'event_ids': '2949650',
    'attendance_status': 'attended',
    'attendance_operator': 'any',
    'page': 'spreadsheet',
}

print("Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.")
print(base_url + urllib.parse.urlencode(params))

Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.
https://newmarketchurch.breezechms.com/events/reports/243292484#/start_date=01%2F01%2F2024&end_date=05%2F28%2F2024&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet


In [2]:
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/events/reports/243292472#/&start_date=10%2F01%2F2023&end_date=10%2F31%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet

attendancereport = r"C:\Users\ralph\Downloads\FoodPantry-attendance-05-27-2024.xlsx"
peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-05-24-2024.xlsx"

import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
attendance = pd.read_excel(attendancereport)

# drop the Total row
attendance = attendance[attendance['Breeze ID'].notna()]

people = pd.read_excel(peopleexport)
print('{count} people in input.'.format(count = len(attendance.index)))

236 people in input.


In [3]:
# Keep only the columns we use.

attendance = attendance.drop([
    'First Name', 
    'Last Name', 
    'Person Totals'
], axis = 1)

dates = attendance.columns[attendance.columns != 'Breeze ID']

family_size = {k: v for k, v in zip(people['Breeze ID'], people['How many people live in your household? (including yourself)'])}


In [4]:
attendance.fillna(0, inplace=True)

people_fed = {}
for date in dates:
    datesum = 0;
    for _, row in attendance.iterrows():
        if row[date] == 'X':
            datesum += family_size[row['Breeze ID']]
    people_fed[date] = datesum



In [5]:
totalFed = sum(people_fed.values())

output = pd.DataFrame({'Date': dates, 
                       'People': people_fed.values(), 
                       'Value': [round(p * donatedFoodValue / totalFed, 2) for p in people_fed.values()] })

In [6]:
output

,Date,People,Value
0,"Jan 4, 2024",216,5739.82
1,"Jan 11, 2024",211,5606.95
2,"Jan 18, 2024",251,6669.88
3,"Jan 25, 2024",285,7573.37
4,"Feb 1, 2024",241,6404.15
5,"Feb 8, 2024",274,7281.07
6,"Feb 15, 2024",225,5978.98
7,"Feb 22, 2024",250,6643.31
8,"Feb 29, 2024",212,5633.53
9,"Mar 7, 2024",231,6138.42
